In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
import matplotlib.pylab as plt

In [2]:
def SVI(moneyness,a,b,rho,m,s):
    return np.sqrt(a+b*(rho*(np.log(moneyness)-m)+np.sqrt((np.log(moneyness)-m)**2+s**2)))

In [15]:
df = pd.read_csv("D:\\500option_ L2_options_20020301.csv")
df['Pos'] = df['UnderlyingPrice'] - df['Strike']
df = df[np.logical_or(np.logical_and(df['Pos']>0,df['Type']=='put'),np.logical_and(df['Pos']<0,df['Type']=='call'))]
df = df.reset_index(drop=True)

In [16]:
stocks = df['UnderlyingSymbol'].unique()
vol_paras = []
initial = 0.1,0.1,0.1,0.1,0.1
ctr = 0
for stock in stocks[:10]:
    df_stock = df[df['UnderlyingSymbol']==stock]
    maturities = df_stock['Expiration'].unique()
    S = df_stock['UnderlyingPrice'].max()
    ctr += 1
    for day in maturities:
        strikes = np.array(df_stock[df_stock['Expiration']==day]['Strike'])
        iv = np.array(df_stock[df_stock['Expiration']==day]['IV'])
        try:
            para,_ = curve_fit(SVI, strikes/S, iv, initial, bounds = ((0,0,-1,-np.inf,0),(np.inf,np.inf,1,np.inf, np.inf)), maxfev = 5000)
        except:
            para = [0,0,0,0,0]
        vol_paras += [[day,stock]+list(para)+[strikes]+[iv]]

C:\Users\ttkad\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


In [19]:
vol_paras = pd.DataFrame(vol_paras,columns=['Maturity','Ticker','a','b','rho','m','s','stikes','iv'])
vol_paras

,Maturity,Ticker,a,b,rho,m,s,stikes,iv
0,03/16/2002,GE,1.781041e-01,0.242703,-0.962150,0.061625,1.240764e-02,"[25.0, 27.5, 30.0, 32.5, 35.0, 37.5, 40.0, 42....","[0.6119, 0.6119, 0.6119, 0.6119, 0.5119, 0.423..."
1,04/20/2002,GE,6.874060e-02,0.178390,-1.000000,-0.006177,1.179731e-01,"[27.5, 30.0, 32.5, 35.0, 37.5, 40.0, 42.5, 45....","[0.4315, 0.4315, 0.3783, 0.3369, 0.3073, 0.302..."
2,06/22/2002,GE,1.788548e-19,0.256302,0.021524,0.173578,2.614357e-01,"[25.0, 27.5, 30.0, 32.5, 35.0, 37.5, 40.0, 42....","[0.4044, 0.4044, 0.364, 0.3308, 0.3047, 0.2856..."
3,09/21/2002,GE,6.168023e-12,0.147079,-1.000000,-0.201288,7.538128e-01,"[27.5, 30.0, 32.5, 35.0, 37.5, 40.0, 42.5, 45....","[0.3701, 0.3497, 0.331, 0.3142, 0.2991, 0.2938..."
4,01/18/2003,GE,1.690343e-02,0.288185,0.683278,0.585223,2.112844e-01,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55....","[0.3626, 0.3626, 0.3163, 0.2822, 0.2937, 0.270..."
5,01/17/2004,GE,1.948856e-13,0.229282,0.602166,0.714248,3.824825e-01,"[20.0, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 60....","[0.3765, 0.3458, 0.3217, 0.3041, 0.3005, 0.283..."
6,03/16/2002,MSFT,1.503850e-01,0.170175,-0.999889,0.198271,6.043213e-15,"[30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60.0, 65....","[0.6176, 0.6176, 0.6176, 0.6176, 0.6176, 0.522..."
7,04/20/2002,MSFT,9.819635e-02,0.132608,-0.322943,0.133765,2.517163e-11,"[25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60....","[0.4807, 0.4807, 0.4807, 0.4807, 0.4807, 0.413..."
8,07/20/2002,MSFT,6.875103e-02,0.197942,0.368130,0.333455,7.216218e-02,"[25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55.0, 60....","[0.4412, 0.4412, 0.4412, 0.4412, 0.4019, 0.370..."
9,10/19/2002,MSFT,8.238564e-02,0.099567,-0.222221,0.299240,3.145469e-02,"[22.5, 25.0, 30.0, 35.0, 40.0, 45.0, 50.0, 55....","[0.4646, 0.4646, 0.4646, 0.4646, 0.4299, 0.400..."


In [10]:
df = pd.read_csv("D:\\500option_L2_options_20020301_Vol_Surf.csv")
a = df['vol_surf'][0]